# 1 Imports

In [1]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

# 2 Configuration

In [2]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPEN_API_KEY')

In [3]:
# initialize

openai = OpenAI()
openai_model = "gpt-4o"

# 3 Code Generator

## 3.1 Functions Configuration

In [4]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac."
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments."
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [5]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time."
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments."
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packagtes such as iomanip. \n\n"
    user_prompt += python
    return user_prompt

In [6]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [7]:
# Create a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [8]:
def optimize_gpt(python):
    
    stream = openai.chat.completions.create(model=openai_model, messages=messages_for(python), stream=True)
    reply = ""
    
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end = "", flush=True)
    
    write_output(reply)

## 3.2 Code Translation

In [9]:
pi = """
import time

def calculate(interations, param1, param2):
    result = 1.0

    for i in range(1, interations + 1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [10]:
# Check code
exec(pi)

Result: 3.141592658589
Execution Time: 8.698939 seconds


In [11]:
# Convert the code to C++
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

inline double calculate(long long iterations, int param1, int param2) {
    double result = 1.0;

    for (long long i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100'000'000LL, 4, 1) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::chrono::duration<double> execution_time = end_time - start_time;

    std::cout << std::fixed << std::setprecision(12) << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << std::fixed << std::setprecision(6)
              << execution_time.count() << " seconds" << std::endl;

    return 0;
}
```

## 3.4 Gradio UI

In [12]:
def stream_gpt_4(python):
    stream = openai.chat.completions.create(model=openai_model, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace("```cpp\n", "").replace("```", "")
    print(fragment)

In [13]:
def stream_gpt_35(python):
    stream = openai.chat.completions.create(model="gpt-3.5-turbo", messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace("```cpp", "").replace("```", "")

In [ ]:
def optimize(python, model):

    if model == "gpt-4":
        result = stream_gpt_4(python)
    elif model == "gpt-3.5":
        result = stream_gpt_35(python)
    else:
        raise ValueError("Invalid model")
    
    for stream_so_far in result:
        yield stream_so_far

In [17]:
with gr.Blocks() as ui:

    with gr.Row():
        python = gr.Textbox(label='Python code:', lines=10, value=pi)
        cpp = gr.Textbox(label='C++ code:', lines=10)
    
    with gr.Row():
        model = gr.Dropdown(["gpt-4", "gpt-3.5"], label="Model", value="gpt-4")
        convert = gr.Button('Convert code')

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
